In [1]:
import pandas as pd
import math

CLASSES = 2 # min of binary classification

In [2]:
def get_entropy(data,c_label): # pass data and class label name
    total = len(data[c_label]) # total number of samples
    entropy = 0 # set up entropy to sum up each labels probability
    labels = data[c_label].unique()
    for l in labels: # iterate over labels
        count = data[c_label].isin([l]).sum(axis=0)
        entropy += (-count/total) * math.log((count/total),CLASSES)
    return entropy

In [3]:
def get_information_gain(data,f_label,c_label): # pass data, feature label, and class label name
     total = len(data[f_label]) # total number of samples
     gain = get_entropy(data,c_label) # setup gain as current entropy
     values = data[f_label].unique()
     for v in values: # iterate over feature values
        count = data[f_label].isin([v]).sum(axis=0)
        gain -= count/total * get_entropy(data[data[f_label].isin([v])],c_label)
     return gain
    
    
    # info_gain = get_entropy(data, c_label)
    # for f in f_label:
    #     info_gain -= (f/c_label)*get_entropy(data,f)

In [4]:
def build_tree(data,features,c_label,T):
    if len(data) == 0 or len(features) == 0:
        return None # end condition: no data or features left to check on this branch!

    info_gain = {}
    for f in features: # iterate over features
        info_gain[f] = get_information_gain(data,f,c_label)

    if max(info_gain.values()) == 0:
        return None # end condition: nothing more to learn!

    pick = max(info_gain, key=info_gain.get) # argmax
    branches = {}
    for v in data[pick].unique():
        branches[v] = {}
        node = build_tree(data[data[pick].isin([v])],features,c_label,branches[v])
    T[pick] = branches
    return pick

In [5]:
# this function courtesy of ChatGPT:
def print_tree(tree, indent=''):
    if not tree:
        return
    if isinstance(tree, dict):
        for key, subtree in tree.items():
            print(indent + str(key))
            print_tree(subtree, indent + '  ')
    else:
        print(indent + str(tree))

In [6]:
df = pd.read_csv('golf.csv')
c_label = 'Play Golf'
CLASSES = len(df[c_label].unique())

features = df.columns.values.tolist()
features.remove(c_label)

T = {}
build_tree(df,features,c_label,T)
print_tree(T)

Outlook
  Sunny
    Humidity
      High
      Normal
  Overcast
  Rain
    Wind
      Weak
      Strong
